# Day 1

In [1]:
import re, timeit, random

## Chapter 3

In [2]:
class Stack:
    def __init__(self):
        self.items = []

    def isEmpty(self):
        return self.items == []

    def push(self, item):
        self.items.append(item)

    def pop(self):
        return self.items.pop()

    def peek(self):
        return self.items[len(self.items) - 1]

    def size(self):
        return len(self.items)

In [11]:
def parChecker(symbolString):
    s = Stack()
    balanced = True
    index = 0
    while index < len(symbolString) and balanced:
        symbol = symbolString[index]
        if symbol == "(":
            s.push(symbol)
        elif symbol == ")":
            if s.isEmpty():
                balanced = False
            else:
                s.pop()

        index += 1

    return (balanced and s.isEmpty())


def fixSpacing(inputString):
    """
    Convert poorly-spaced string to a better-spaced one.

    :param inputString:
    :return:
    """
    tokenList = []
    for token in inputString:
        if token != " ":
            tokenList.append(token)

    return tokenList


def popAndCompute(operandStack, op):
    def doMath(op, op1, op2):
        try:
            if op == "*":
                return op1 * op2
            elif op == "/":
                return op1 / op2
            elif op == "+":
                return op1 + op2
            elif op == "-":
                return op1 + -op2
            elif op == "^":
                return op1 ** op2
        except KeyError:
            print("{} not recognized.".format(op))
            raise KeyError

    operand2 = operandStack.pop()
    operand1 = operandStack.pop()
    result = doMath(op, operand1, operand2)
    operandStack.push(result)

### Exercise 1

"Modify the postfix evaluation algorithm so that it can handle errors."

In [4]:
def infixToPostfix(infixexpr):
    prec = {
        "^": 4,
        "*": 3,
        "/": 3,
        "+": 2,
        "-": 2,
        "(": 1,
    }
    opStack = Stack()
    postfixList = []

    # ensure there are equal numbers of left and right parentheses
    if not parChecker(infixexpr):
        return "Mismatched parentheses."

    tokenList = fixSpacing(infixexpr)

    # actual implementation
    for token in tokenList:
        if token in "ABCDEFGHIJKLMNOPQRSTUVWXYZ" or token in "0123456789":
            postfixList.append(token)
        elif token == '(':
            opStack.push(token)
        elif token == ')':
            topToken = opStack.pop()
            while topToken != '(':
                postfixList.append(topToken)
                topToken = opStack.pop()
        else:
            while (not opStack.isEmpty()) and \
                    (prec[opStack.peek()] >= prec[token]):
                postfixList.append(opStack.pop())
            opStack.push(token)

    while not opStack.isEmpty():
        postfixList.append(opStack.pop())
    return " ".join(postfixList)

In [5]:
print(infixToPostfix("A * B + C * D"))
print(infixToPostfix("A*B+C*D"))
print(infixToPostfix("( A + B ) * C - ( D - E ) * ( F + G )"))
print(infixToPostfix("(A+B) * C - (D - E )* ( F +G)"))
print(infixToPostfix("( A + B ) * C - ( D - E )) * ( F + G )"))
print(infixToPostfix("( A + B ) * (C - ( D - E ) * ( F + G )"))

A B * C D * +
A B * C D * +
A B + C * D E - F G + * -
A B + C * D E - F G + * -
Mismatched parentheses.
Mismatched parentheses.


### Exercise 2

"Modify the postfix evaluation algorithm so that it can handle errors."

In [26]:
def postfixEval(postfixExpr):
    operandStack = Stack()

    tokenList = fixSpacing(postfixExpr)

    for token in tokenList:
        if token in "0123456789":
            operandStack.push(int(token))
        elif token in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
            print("Variables like '{}' are currently not supported at this time.".format(token))
            raise KeyError
        else:
            popAndCompute(operandStack, token)
    return operandStack.pop()

In [27]:
print(postfixEval('7 8 + 3 2 + /'))
print(postfixEval('7 8+3 2 + /'))
# print(postfixEval('7 8 + 3 A + /'))

3.0
3.0


### Exercise 3

"Implement a direct infix evaluator that combines the functionality of infix-to-postfix conversion and the postfix
evaluation algorithm. Your evaluator should process infix tokens from left to right and use two stacks, one for
operators and one for operands, to perform the evaluation."

In [13]:
def infixToPostfixEval(infixexpr):
    prec = {
        "^": 4,
        "*": 3,
        "/": 3,
        "+": 2,
        "-": 2,
        "(": 1,
    }
    operatorStack = Stack()
    operandStack = Stack()

    # ensure there are equal numbers of left and right parentheses
    if not parChecker(infixexpr):
        return "Mismatched parentheses."

    tokenList = fixSpacing(infixexpr)

    # actual implementation
    for token in tokenList:
        if token in "0123456789":
            operandStack.push(int(token))
        elif token == '(':
            operatorStack.push(token)
        elif token == ')':
            topToken = operatorStack.pop()
            while topToken != '(':
                popAndCompute(operandStack, topToken)
                topToken = operatorStack.pop()
        else:
            while (not operatorStack.isEmpty()) and \
                    (prec[operatorStack.peek()] >= prec[token]):
                popAndCompute(operandStack, operatorStack.pop())
            else:
                operatorStack.push(token)

    while not operatorStack.isEmpty():
        popAndCompute(operandStack, operatorStack.pop())

    return operandStack.pop()

In [14]:
print(infixToPostfixEval("2 * 2 + 3 * 3"))
print(infixToPostfixEval("( 3 + 2 ) * 5 - ( 4 - 3 ) * ( 8 + 2 )"))

13
15


### Exercise 4

"Turn your direct infix evaluator from the previous problem into a calculator."

In [ ]:
print("Enter an equation below. Type 'quit' to exit.")
equation = ""
previous_answer = None

while equation.lower() != "666":
    equation = input("> ")
    if not re.match(r'^[0123456789(]', equation):  # this is adding to the previous equation.
        equation = str(previous_answer) + " " + equation

    previous_answer = infixToPostfixEval(equation)
    print(previous_answer)

### Exercise 5

"Implement the Queue ADT, using a list such that the rear of the queue is at the end of the list."

In [17]:
class Queue:
    def __init__(self):
        self.items = []

    def isEmpty(self):
        return self.items == []

    def enqueue(self, item):
        self.items.append(item)

    def dequeue(self):
        return self.items.pop(0)

    def size(self):
        return len(self.items)


q = Queue()
print(q.isEmpty())
q.enqueue(4)
q.enqueue('dog')
q.enqueue(True)
print(q.size())
print(q.isEmpty())
q.enqueue(8.4)
q.dequeue()
q.dequeue()
print(q.size())

True
3
False
2


## Chapter 5

### Exercise 1

In [18]:
def sequentialSearch(list, item):
    pos = 0
    found = False

    while pos < len(list) and not found:
        if list[pos] == item:
            found = True
        else:
            pos += 1

    return found


def binarySearch(list, item):
    first = 0
    last = len(list) - 1
    found = False

    while first <= last and not found:
        midpoint = (first + last) // 2
        if list[midpoint] == item:
            found = True
        else:
            if item < list[midpoint]:
                last = midpoint - 1
            else:
                first = midpoint + 1

    return found

In [19]:
testlist = [1, 2, 32, 8, 17, 19, 42, 13, 0]
print(sequentialSearch(testlist, 3))
print(sequentialSearch(testlist, 13))

testlist = [0, 1, 2, 8, 13, 17, 19, 32, 42, 56, 60, 66, 72, 74, 80, 81, 82, 83, 99,]
print(binarySearch(testlist, 3))
print(binarySearch(testlist, 13))

print(timeit.repeat('sequentialSearch(testlist, 83)', 'from __main__ import sequentialSearch, testlist', number=10000))
print(timeit.repeat('binarySearch(testlist, 83)', 'from __main__ import binarySearch, testlist', number=10000))

False
True
False
True
[0.03495569900042028, 0.024239533000582014, 0.022364368000125978]
[0.007712327998888213, 0.007820104001439176, 0.007468083000276238]


### Exercise 2

"Use the binary search functions given in the text (recursive and iterative). Generate a random, ordered list of
integers and do a benchmark analysis for each one. What are your results? Can you explain them?"

In [20]:
def binarySearchIterative(list, item):
    first = 0
    last = len(list) - 1
    found = False

    while first <= last and not found:
        midpoint = (first + last) // 2
        if list[midpoint] == item:
            found = True
        else:
            if item < list[midpoint]:
                last = midpoint - 1
            else:
                first = midpoint + 1

    return found


def binarySearchRecursive(list, item):
    if len(list) == 0:
        return False
    else:
        midpoint = len(list) // 2
        if list[midpoint] == item:
            return True
        else:
            if item < list[midpoint]:
                return binarySearchRecursive(list[:midpoint], item)
            else:
                return binarySearchRecursive(list[midpoint + 1:], item)


def generateRandomList(length):
    returnlist = []
    most_recent = 0
    for _ in range(length):
        randomentry = random.randint(most_recent + 1, most_recent + 10)
        most_recent = randomentry
        returnlist.append(randomentry)
    return returnlist

In [21]:
testlist = generateRandomList(1000)

max_value = testlist[len(testlist) - 1]

print(timeit.repeat("binarySearchIterative(testlist, random.randint(0, max_value))",
                  'from __main__ import binarySearchIterative, random, testlist, max_value',
                  number=100000))

print(timeit.repeat("binarySearchRecursive(testlist, random.randint(0, max_value))",
                        'from __main__ import binarySearchRecursive, random, testlist, max_value',
                        number=100000))

[0.35115421999944374, 0.32775948500056984, 0.32633111200084386]
[0.7905835850015137, 0.9512102339995181, 0.9342387039996538]


In general, the recursive method takes more than twice as long to complete than the iterative one. This is because the recursive one must complete a split on the lists during each iteration. The splits in total add up to an O(n) addition per run of the algorithm, turning an O(logn) algorithm to an O(nlogn) algorithm. In comparison, the iterative method takes only O(logn) to run, since it only takes one run through the list to find a value. Exercise 3 will prove this by building a new recusive method that should perform equally well to the iterative method.

Iterative times: [0.376329767000243, 0.37340135500016913, 0.3594461990005584]

Recursive times: [0.8323799640002107, 0.8352801569999428, 0.8302294299992354]

### Exercise 3

Implement the binary search using recursion without the slice operator. Recall that you will need to pass the list along with the starting and ending index values for the sublist. Generate a random, ordered list of integers and do a benchmark analysis.

In [22]:
def binarySearchRecursive(list, item, startindex, endindex):
    if endindex - startindex == 0:
        return False
    else:
        midpoint = (endindex - startindex) // 2 + startindex
        if list[midpoint] == item:
            return True
        else:
            if item < list[midpoint]:
                return binarySearchRecursive(list, item, startindex, midpoint)
            else:
                return binarySearchRecursive(list, item, midpoint + 1, endindex)


def generateRandomList(length):
    returnlist = []
    most_recent = 0
    for _ in range(length):
        randomentry = random.randint(most_recent + 1, most_recent + 10)
        most_recent = randomentry
        returnlist.append(randomentry)
    return returnlist

In [23]:
testlist = generateRandomList(1000)

max_value = testlist[len(testlist) - 1]

print(timeit.repeat("binarySearchRecursive(testlist, random.randint(0, max_value), 0, len(testlist) - 1)",
                    'from __main__ import binarySearchRecursive, random, testlist, max_value',
                    number=100000))

[0.43521571199926257, 0.4209791569992376, 0.4263306350003404]


Implement the binary search using recursion without the slice operator. Recall that you will need to pass the list along with the starting and ending index values for the sublist. Generate a random, ordered list of integers and do a benchmark analysis.

### Exercise 4 and 5

"Implement the len method (__len__) for the hash table Map ADT implementation."
"Implement the in method (__contains__) for the hash table Map ADT implementation."

In [24]:
class HashTable:
    def __init__(self):
        self.size = 11
        self.slots = [None] * self.size
        self.data = [None] * self.size

    def put(self, key, data):
        hashvalue = self.hashfunction(key, len(self.slots))

        if self.slots[hashvalue] is None:
            self.slots[hashvalue] = key
            self.data[hashvalue] = data
        else:
            if self.slots[hashvalue] == key:
                self.data[hashvalue] = data  # replace
            else:
                nextslot = self.rehash(hashvalue, len(self.slots))
                while self.slots[nextslot] is not None and \
                    self.slots[nextslot] != key:
                    nextslot = self.rehash(nextslot, len(self.slots))

                if self.slots[nextslot] is None:
                    self.slots[nextslot] = key
                    self.data[nextslot] = data
                else:
                    self.data[nextslot] = data  # replace

    def hashfunction(self, key, size):
        return key % size

    def rehash(self, oldhash, size):
        return (oldhash + 1) % size

    # Exercise 4
    def __len__(self):
        count = 0
        for i in self.slots:
            if i is not None:
                count += 1
        return count

    # Exercise 5
    def __contains__(self, item):
        hashvalue = self.hashfunction(item, len(self.slots))

        if self.slots[hashvalue] is None:
            return False
        else:
            if self.slots[hashvalue] == item:
                return True
            else:
                nextslot = self.rehash(hashvalue, len(self.slots))
                while self.slots[nextslot] is not None and \
                    self.slots[nextslot] != item:
                    nextslot = self.rehash(nextslot, len(self.slots))

                return self.slots[nextslot] == item

In [25]:
h = HashTable()

h.put(1, 'cat')
h.put(16, 'fish')
h.put(12, 'elephant')
h.put(4, 'rhino')

print(len(h))
print(1 in h)
print(3 in h)


4
True
False
